In [210]:
import pandas as pd 

df = pd.read_excel("Online_Retail.xlsx")

In [211]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [212]:
df = df.dropna(subset=['CustomerID'])

In [213]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [214]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [215]:
import datetime as dt

def helper_date(mn):
    return dt.datetime(mn.year, mn.month, 1)

df['InvoiceMonth'] = df['InvoiceDate'].apply(helper_date)

In [216]:
# create a column index with the minimum invoice date aka first time customer was acquired
df['CohortMonth'] = df.groupby('CustomerID')['InvoiceMonth'].transform(min)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth,CohortMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01


In [217]:
def get_date_element(df, col):

    day = df[col].dt.day
    month = df[col].dt.month
    year = df[col].dt.year

    return day, month, year

In [218]:
_, InvoiceMonth, InvoiceYear = get_date_element(df, 'InvoiceMonth')
_, CohortMonth, CohortYear = get_date_element(df, 'CohortMonth')

In [219]:
# create cohort index
year_diff = InvoiceYear - CohortYear
month_diff = InvoiceMonth - CohortMonth
df['CohortIndex'] = year_diff * 12 + month_diff + 1

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth,CohortMonth,CohortIndex
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,2010-12-01,1
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01,1
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,2010-12-01,1
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01,1
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01,1


In [220]:
cohort_data = df.groupby(['CohortMonth', 'CohortIndex'])['CustomerID'].apply(pd.Series.nunique).reset_index()
cohort_data.head()

,CohortMonth,CohortIndex,CustomerID
0,2010-12-01,1,948
1,2010-12-01,2,362
2,2010-12-01,3,317
3,2010-12-01,4,367
4,2010-12-01,5,341


In [221]:
# Customer cohort

customer_cohort = cohort_data.pivot(index='CohortMonth', columns='CohortIndex', values='CustomerID')

# Total number of customer on the first month of that cohort like 2010-12-01 - 948 is the cohort size of that month
cohort_sizes = customer_cohort.iloc[:, 0]

customer_cohort.fillna('')

CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2010-12-01,948.0,362.0,317.0,367.0,341.0,376.0,360.0,336.0,336.0,374.0,354.0,474.0,260.0
2011-01-01,421.0,101.0,119.0,102.0,138.0,126.0,110.0,108.0,131.0,146.0,155.0,63.0,
2011-02-01,380.0,94.0,73.0,106.0,102.0,94.0,97.0,107.0,98.0,119.0,35.0,,
2011-03-01,440.0,84.0,112.0,96.0,102.0,78.0,116.0,105.0,127.0,39.0,,,
2011-04-01,299.0,68.0,66.0,63.0,62.0,71.0,69.0,78.0,25.0,,,,
2011-05-01,279.0,66.0,48.0,48.0,60.0,68.0,74.0,29.0,,,,,
2011-06-01,235.0,49.0,44.0,64.0,58.0,79.0,24.0,,,,,,
2011-07-01,191.0,40.0,39.0,44.0,52.0,22.0,,,,,,,
2011-08-01,167.0,42.0,42.0,42.0,23.0,,,,,,,,


In [222]:
# Calculate retention rate as percentage

cohort_counts = cohort_data.pivot(index='CohortMonth', columns='CohortIndex', values='CustomerID')

cohort_retention = (cohort_counts.divide(cohort_sizes, axis=0) * 100).round(2).fillna('')
cohort_retention


CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2010-12-01,100.0,38.19,33.44,38.71,35.97,39.66,37.97,35.44,35.44,39.45,37.34,50.0,27.43
2011-01-01,100.0,23.99,28.27,24.23,32.78,29.93,26.13,25.65,31.12,34.68,36.82,14.96,
2011-02-01,100.0,24.74,19.21,27.89,26.84,24.74,25.53,28.16,25.79,31.32,9.21,,
2011-03-01,100.0,19.09,25.45,21.82,23.18,17.73,26.36,23.86,28.86,8.86,,,
2011-04-01,100.0,22.74,22.07,21.07,20.74,23.75,23.08,26.09,8.36,,,,
2011-05-01,100.0,23.66,17.2,17.2,21.51,24.37,26.52,10.39,,,,,
2011-06-01,100.0,20.85,18.72,27.23,24.68,33.62,10.21,,,,,,
2011-07-01,100.0,20.94,20.42,23.04,27.23,11.52,,,,,,,
2011-08-01,100.0,25.15,25.15,25.15,13.77,,,,,,,,


In [223]:
# Net Reveue calculation

df['NetRevenue'] = df['Quantity'] * df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth,CohortMonth,CohortIndex,NetRevenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,2010-12-01,1,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01,1,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,2010-12-01,1,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01,1,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01,1,20.34


In [224]:
# Net Reveue Cohort

cohort_net_revenue = df.pivot_table(index='CohortMonth', columns='CohortIndex', values='NetRevenue', aggfunc='sum')

cohort_net_revenue = cohort_net_revenue.round(2)

cohort_net_revenue.fillna('')

CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2010-12-01,554604.02,271616.52,230856.22,302509.39,200927.98,321611.71,312894.36,303907.31,310882.67,466580.68,438835.91,510982.23,182814.95
2011-01-01,203457.86,56168.55,62264.88,41516.17,82313.31,84009.54,70218.5,72886.27,74430.63,104181.16,122027.2,27850.51,
2011-02-01,149521.38,25260.26,37456.25,45852.34,35666.69,31081.74,47665.32,55840.13,51861.55,60544.86,9366.67,,
2011-03-01,189930.08,26383.2,53693.95,40324.05,46759.97,38360.36,60622.28,61705.53,65303.87,11252.28,,,
2011-04-01,119764.25,28921.03,24928.06,23892.08,26006.54,29653.29,28223.83,33889.78,6292.0,,,,
2011-05-01,115858.74,17663.95,18876.66,17888.89,26591.53,32951.52,31401.9,10629.02,,,,,
2011-06-01,92526.51,13619.35,13893.45,29925.03,25835.74,39764.29,7925.61,,,,,,
2011-07-01,65882.87,11126.27,15395.71,17222.51,19329.06,6056.16,,,,,,,
2011-08-01,77658.23,19122.5,33061.56,39925.01,14143.23,,,,,,,,


In [225]:
# Calculate retention rate as percentage

cohort_net_revenue_perc = (cohort_net_revenue.divide(cohort_sizes, axis=0) * 100).round(2)

cohort_net_revenue_perc.fillna('')

CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2010-12-01,58502.53,28651.53,24351.92,31910.27,21194.93,33925.29,33005.73,32057.73,32793.53,49217.37,46290.71,53901.08,19284.28
2011-01-01,48327.28,13341.7,14789.76,9861.32,19551.86,19954.76,16678.98,17312.65,17679.48,24746.12,28985.08,6615.32,
2011-02-01,39347.73,6647.44,9856.91,12066.41,9385.97,8179.41,12543.51,14694.77,13647.78,15932.86,2464.91,,
2011-03-01,43165.93,5996.18,12203.17,9164.56,10627.27,8718.26,13777.79,14023.98,14841.79,2557.34,,,
2011-04-01,40054.93,9672.59,8337.14,7990.66,8697.84,9917.49,9439.41,11334.37,2104.35,,,,
2011-05-01,41526.43,6331.16,6765.83,6411.79,9531.01,11810.58,11255.16,3809.68,,,,,
2011-06-01,39372.98,5795.47,5912.11,12734.06,10993.93,16920.97,3372.6,,,,,,
2011-07-01,34493.65,5825.27,8060.58,9017.02,10119.93,3170.76,,,,,,,
2011-08-01,46501.93,11450.6,19797.34,23907.19,8469.0,,,,,,,,


In [226]:
# Cummulative lifetime value (prefix sum)

cumulative_ltv = cohort_net_revenue.cumsum(axis=1)

cumulative_ltv = cumulative_ltv.round(2)
cumulative_ltv.fillna('')


CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2010-12-01,554604.02,826220.54,1057076.76,1359586.15,1560514.13,1882125.84,2195020.2,2498927.51,2809810.18,3276390.86,3715226.77,4226209.0,4409023.95
2011-01-01,203457.86,259626.41,321891.29,363407.46,445720.77,529730.31,599948.81,672835.08,747265.71,851446.87,973474.07,1001324.58,
2011-02-01,149521.38,174781.64,212237.89,258090.23,293756.92,324838.66,372503.98,428344.11,480205.66,540750.52,550117.19,,
2011-03-01,189930.08,216313.28,270007.23,310331.28,357091.25,395451.61,456073.89,517779.42,583083.29,594335.57,,,
2011-04-01,119764.25,148685.28,173613.34,197505.42,223511.96,253165.25,281389.08,315278.86,321570.86,,,,
2011-05-01,115858.74,133522.69,152399.35,170288.24,196879.77,229831.29,261233.19,271862.21,,,,,
2011-06-01,92526.51,106145.86,120039.31,149964.34,175800.08,215564.37,223489.98,,,,,,
2011-07-01,65882.87,77009.14,92404.85,109627.36,128956.42,135012.58,,,,,,,
2011-08-01,77658.23,96780.73,129842.29,169767.3,183910.53,,,,,,,,


In [227]:
# Customer lifetime revenue

clr = cumulative_ltv.divide(cohort_sizes, axis=0).round(2)

clr.fillna('')

CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2010-12-01,585.03,871.54,1115.06,1434.16,1646.11,1985.36,2315.42,2636.0,2963.93,3456.11,3919.02,4458.03,4650.87
2011-01-01,483.27,616.69,764.59,863.2,1058.72,1258.27,1425.06,1598.18,1774.98,2022.44,2312.29,2378.44,
2011-02-01,393.48,459.95,558.52,679.18,773.04,854.84,980.27,1127.22,1263.7,1423.03,1447.68,,
2011-03-01,431.66,491.62,613.65,705.3,811.57,898.75,1036.53,1176.77,1325.19,1350.76,,,
2011-04-01,400.55,497.28,580.65,660.55,747.53,846.71,941.1,1054.44,1075.49,,,,
2011-05-01,415.26,478.58,546.23,610.35,705.66,823.77,936.32,974.42,,,,,
2011-06-01,393.73,451.68,510.81,638.15,748.09,917.3,951.02,,,,,,
2011-07-01,344.94,403.19,483.8,573.97,675.16,706.87,,,,,,,
2011-08-01,465.02,579.53,777.5,1016.57,1101.26,,,,,,,,


In [228]:
# Customer lifetime value

gross_margin = 0.60 # 60%
CAC = 50 # Customer acquisition cost

clv = (clr * gross_margin) - CAC

clv.round(2).fillna('')

CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2010-12-01,301.02,472.92,619.04,810.5,937.67,1141.22,1339.25,1531.6,1728.36,2023.67,2301.41,2624.82,2740.52
2011-01-01,239.96,320.01,408.75,467.92,585.23,704.96,805.04,908.91,1014.99,1163.46,1337.37,1377.06,
2011-02-01,186.09,225.97,285.11,357.51,413.82,462.9,538.16,626.33,708.22,803.82,818.61,,
2011-03-01,209.00,244.97,318.19,373.18,436.94,489.25,571.92,656.06,745.11,760.46,,,
2011-04-01,190.33,248.37,298.39,346.33,398.52,458.03,514.66,582.66,595.29,,,,
2011-05-01,199.16,237.15,277.74,316.21,373.4,444.26,511.79,534.65,,,,,
2011-06-01,186.24,221.01,256.49,332.89,398.85,500.38,520.61,,,,,,
2011-07-01,156.96,191.91,240.28,294.38,355.1,374.12,,,,,,,
2011-08-01,229.01,297.72,416.5,559.94,610.76,,,,,,,,
